In [1]:
# simple_fuse: simple fusion from acoustic and txt
# run in MPC as : exec(open('simple_fuse_dev.py').read())
# 2019-12-20: initial work, it works!
# 2019-12-21: update to use silence feature from speech network
#             new data splitting (6000/2000/2039)

import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from calc_scores import calc_scores
from sklearn import linear_model
from sklearn.linear_model import Ridge,SGDRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,ConstantKernel as c

# option: ser, ser_hfs, ser_ws, ter, ter_w2v, ter_glove, is saved change the name
# speech = 'ser'           # ser, ser_hfs, ser_ws
# text = 'ter'          # ter, ter_w2v, ter_glove'
ser = np.load('../speech/npy7869/speech_7869-npy15.npy')
ter = np.load('../text/npy7869/text_npy-7869glove4.npy')
mocap = np.load('../MOCAP/npy7869/mocap_npy-7869batch1.npy')
st = np.load('../speech+text/npy7869/speech_text_npy-7869glove4.npy')
sm = np.load('../speech+mocap/npy7869/speech_mocap_npy-7869batch11.npy')
tm = np.load('../text+mocap/npy7869/text_mocapGLOVE_7869npy16.npy')
stm = np.load('../speech+text+mocap/npy7869/stm_npy-glove-7869batch7.npy')
#mocap = np.load('../../data/predict_mocap_cnn1.npy')

In [2]:
# split dev and test
split = 1579
ser_dev = ser[:split]
ser_test = ser[split:]
ter_dev = ter[:split]
ter_test = ter[split:]
mocap_dev = mocap[:split]
mocap_test = mocap[split:]
st_dev = st[:split]
st_test = st[split:]
sm_dev = sm[:split]
sm_test = sm[split:]
tm_dev = tm[:split]
tm_test = tm[split:]
stm_dev = stm[:split]
stm_test = stm[split:]
# load label 
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# #remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) 
    scaled_vad = scaler.transform(vad) 
    vad = scaled_vad 
else:
    vad = vad

y_dev = vad[6290:7869]
y_test = vad[7869:] 

In [3]:
# SVR model
svr_rbf = SVR(kernel='rbf')
#svr_rbf = linear_model.RidgeCV(fit_intercept=True)

#svr_rbf = GaussianProcessRegressor(n_restarts_optimizer=10,alpha=0.1)
#svr_rbf = SVC(kernel='rbf', C=200, gamma=0.01,tol=0.01)
#svr_rbf = svm.SVC(c=0.8,kernel='linear',gamma=20,decision_function_shape='ovr')
# svr_rbf = SGDRegressor(loss="epsilon_insensitive")

# predicting valence
valence_model = svr_rbf.fit(np.array([ ser_dev[:,0],ter_dev[:,0],mocap_dev[:,0]]).T, y_dev[:,0])
valence_pred = valence_model.predict(np.array([ ser_test[:,0],ter_test[:,0],mocap_test[:,0]]).T)
# a_test = np.array([mocap_dev[:,0],ter_dev[:,0],ser_dev[:,0]]).T
ccc_v, pcc_v, rmse_v = calc_scores(valence_pred, y_test[:,0])

# predicting arousal
arousal_model = svr_rbf.fit(np.array([ ser_dev[:,1],ter_dev[:,1],mocap_dev[:,1]]).T, y_dev[:,1])
arousal_pred = arousal_model.predict(np.array([ ser_test[:,1],ter_test[:,1],mocap_test[:,1]]).T)
ccc_a, pcc_a, rmse_a = calc_scores(arousal_pred, y_test[:,1])

# predicting dominance
dominance_model = svr_rbf.fit(np.array([ser_dev[:,2],ter_dev[:,2],mocap_dev[:,2]]).T, y_dev[:,2])
dominance_pred = dominance_model.predict(np.array([ ser_test[:,2],ter_test[:,2],mocap_test[:,2]]).T)
ccc_d, pcc_d, rmse_d = calc_scores(dominance_pred, y_test[:,2])

vad_pred = np.vstack((valence_pred, arousal_pred, dominance_pred)).T
# filename = 'speech+text_mocap1.npy' 
# np.save(filename, vad_pred)

In [4]:
#SVR-S+T+M(late)
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6668851961999084 0.6050473302336243 0.5050950404877245
PCC 0.7231322923143084 0.6764649760611002 0.5496857753605642
RMSE 0.2888759059679178 0.2548861945959053 0.2996194153705503


In [15]:
#SVR-S+T+M+STM
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6840043181679756 0.6243299636617339 0.5058370514302154
PCC 0.7322007723628521 0.6813515342674911 0.5508207602669876
RMSE 0.28531283481564207 0.2556092294965971 0.29971464232708944


In [8]:
#SVR-S+M=52.79
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.4844684621871876 0.6176527572345525 0.4813758509789586
PCC 0.5948297653056858 0.6952229643152079 0.519579096302672
RMSE 0.33320709492970046 0.2589268331139633 0.3118629593399738


In [6]:
#SVR-S+M+sm?
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.14830786969531537 0.5829958280561782 0.46441124130045053
PCC 0.19022797413654097 0.6689263029236564 0.5040875903070494
RMSE 0.4836277140085231 0.26437462486016966 0.3166518287392864


In [44]:
#SVR-T+M=46.38
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6480938250807923 0.3755665428123216 0.3677178890831118
PCC 0.6988966484995426 0.484094974221097 0.4828673864399858
RMSE 0.29719478470275235 0.29752412276506396 0.29582105655822427


In [16]:
#SVR-T+M+tm
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.674780479123873 0.38570404930594404 0.352100789591762
PCC 0.7058307743834601 0.5076998141891169 0.45967037109572667
RMSE 0.2917362560591589 0.2932730984364962 0.2999257138061671


In [11]:
#SVR-S+T
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6068583525464696 0.6452897109852593 0.5387407142073093
PCC 0.6644603973487511 0.7130347725001824 0.5725701501834896
RMSE 0.3101936948655447 0.2511827186317671 0.2915918696396121


In [14]:
#SVR-S+T+st
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6137258923271494 0.6467821012361458 0.5497828902650125
PCC 0.6783280400130285 0.7115118877534803 0.586115931769766
RMSE 0.3060616536474912 0.2513636847787265 0.28487624622774327


In [16]:
#SVR-S+T+M
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6453494074043781 0.6417675997808886 0.5205908567322535
PCC 0.6706008721135093 0.705156038362054 0.5493989560636924
RMSE 0.3059636700878019 0.2534788655325252 0.29487989961221844


In [6]:
#SVR-S+T+M+STM
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.706559117203914 0.6407218140306787 0.5278422604078445
PCC 0.7374384432511594 0.6976957221811504 0.5641994708052075
RMSE 0.28618171293468 0.24998550101042633 0.2988843435020853


In [20]:
# SVR model
#svr_rbf = SVR(kernel='rbf', C=200, gamma=0.1, epsilon=0.01)
#svr_rbf = linear_model.RidgeCV(fit_intercept=True)

#svr_rbf = GaussianProcessRegressor(n_restarts_optimizer=10,alpha=0.1)
#svr_rbf = SVC(kernel='rbf', C=200, gamma=0.01,tol=0.01)
#svr_rbf = svm.SVC(c=0.8,kernel='linear',gamma=20,decision_function_shape='ovr')
svr_rbf = SGDRegressor(loss="epsilon_insensitive")

# predicting valence
valence_model = svr_rbf.fit(np.array([ter_dev[:,0],mocap_dev[:,0]]).T, y_dev[:,0])
valence_pred = valence_model.predict(np.array([ter_test[:,0],mocap_test[:,0]]).T)
# a_test = np.array([mocap_dev[:,0],ter_dev[:,0],ser_dev[:,0]]).T
ccc_v, pcc_v, rmse_v = calc_scores(valence_pred, y_test[:,0])

# predicting arousal
arousal_model = svr_rbf.fit(np.array([ter_dev[:,1],mocap_dev[:,1]]).T, y_dev[:,1])
arousal_pred = arousal_model.predict(np.array([ter_test[:,1],mocap_test[:,1]]).T)
ccc_a, pcc_a, rmse_a = calc_scores(arousal_pred, y_test[:,1])

# predicting dominance
dominance_model = svr_rbf.fit(np.array([ter_dev[:,2],mocap_dev[:,2]]).T, y_dev[:,2])
dominance_pred = dominance_model.predict(np.array([ter_test[:,2],mocap_test[:,2]]).T)
ccc_d, pcc_d, rmse_d = calc_scores(dominance_pred, y_test[:,2])

vad_pred = np.vstack((valence_pred, arousal_pred, dominance_pred)).T
# filename = 'speech+text_mocap1.npy' 
# np.save(filename, vad_pred)

In [5]:
#SGD-S+T
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.5525233750217505 0.6231237103850421 0.4910457259997711
PCC 0.6487287805198835 0.7167724232919952 0.5896112263971449
RMSE 0.3177737597596598 0.2517399980023283 0.2871878997256907


In [7]:
#SGD-S+T+ST
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.5744155603154468 0.6372002636293311 0.5111228887998864
PCC 0.6731745279026965 0.7184054131426018 0.6001923233901145
RMSE 0.3109090702596657 0.25043253417289996 0.28385270724694694


In [9]:
#SGD-S+M
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.49770924956467366 0.6245932203793368 0.44953737974439767
PCC 0.5862238763402063 0.707890881951507 0.5467181777402814
RMSE 0.34535660952450736 0.24816144409176044 0.29685096187791216


In [11]:
#SGD-S+M+SM
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.5035421644704067 0.6476061348361546 0.46291634490093486
PCC 0.589838309264654 0.7135366781595741 0.5502941131303215
RMSE 0.33935992718045155 0.2431582063953253 0.2962003041859894


In [21]:
#SGD-T+M
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6445525674504408 0.40955508384368317 0.3077947470681595
PCC 0.7009133596670721 0.5285964593519284 0.49389288002078063
RMSE 0.29611506632287765 0.28962012849780533 0.32163871580331144


In [19]:
#SGD-T+M+TM
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6920412176216599 0.4227649591805766 0.322479739364989
PCC 0.7366101962203592 0.5437625787514051 0.5097575010855782
RMSE 0.283612704792672 0.28704273657338014 0.3137016212329079


In [15]:
#SGD-S+T+M
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6426936095135706 0.6435337451673171 0.48289071969962233
PCC 0.7060388264306081 0.716981701765808 0.5851439982788238
RMSE 0.2981516712977325 0.24446582281520665 0.2884447464092385


In [17]:
#SGD-S+T+M+STM
print('CCC', ccc_v, ccc_a, ccc_d)
print('PCC', pcc_v, pcc_a, pcc_d)
print('RMSE', rmse_v, rmse_a, rmse_d)

CCC 0.6848960669066239 0.6569625003839206 0.5116981829592105
PCC 0.7345495963606122 0.7243016502517874 0.5975252355569345
RMSE 0.2872917202422803 0.24198628745196946 0.2843154732867988
